This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Find the longest absolute file path.

See the [LeetCode](https://leetcode.com/problems/longest-absolute-file-path/) problem page.

<pre>
Suppose we abstract our file system by a string in the following manner:

The string "dir\n\tsubdir1\n\tsubdir2\n\t\tfile.ext" represents:

dir
    subdir1
    subdir2
        file.ext
The directory dir contains an empty sub-directory subdir1 and a sub-directory subdir2 containing a file file.ext.

The string "dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext" represents:

dir
    subdir1
        file1.ext
        subsubdir1
    subdir2
        subsubdir2
            file2.ext

The directory dir contains two sub-directories subdir1 and subdir2. subdir1 contains a file file1.ext and an empty second-level sub-directory subsubdir1. subdir2 contains a second-level sub-directory subsubdir2 containing a file file2.ext.

We are interested in finding the longest (number of characters) absolute path to a file within our file system. For example, in the second example above, the longest absolute path is "dir/subdir2/subsubdir2/file2.ext", and its length is 32 (not including the double quotes).

Given a string representing the file system in the above format, return the length of the longest absolute path to file in the abstracted file system. If there is no file in the system, return 0.

Note:
The name of a file contains at least a . and an extension.
The name of a directory or sub-directory will not contain a .
Time complexity required: O(n) where n is the size of the input string.

Notice that a/aa/aaa/file1.txt is not the longest file path, if there is another path aaaaaaaaaaaaaaaaaaaaa/sth.png.
</pre>

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Is the input a string?
    * Yes
* Can we assume the input is valid?
    * No
* Will there always be a file in the input?
    * Yes
* Is the output an int?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

* None -> TypeError
* '' -> 0
* 'dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext' -> 32

## Algorithm

Refer to the [Solution Notebook]().  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

Solution: parse into a tree (or prefix trie?) and search for the longest string by doing a bfs and storing the current length in the node.

## Code

In [1]:
class TreeNode(object):
    
    def __init__(self, data, parent=None, level=0):
        self.data = data
        self.children = []
        self.parent = parent
        self.level = level
        self.length = len(self.data)
        if self.parent is not None:
            self.length += self.parent.length

    def __repr__(self):
        return 'data:"{}" parent:{} level:{} length:{}'.format(self.data, self.parent.data if self.parent is not None else 'None', self.level, self.length)

class Solution(object):

    def length_longest_path(self, file_system):
        if file_system is None:
            raise TypeError
        if len(file_system) == 0:
            return 0
        s = file_system.split('\n')
        parent = None
        current = None
        for i in s:
            levels = i.split('\t').count('')
            data = i.split('\t')[-1]
            if current is None:
                current = TreeNode(data, level=0, parent=parent)
                continue
            if levels == current.level:
                # same depth as current => same parent.
                current = TreeNode(data, level=levels, parent=parent)
                parent.children.append(current)
            elif levels == current.level+1:
                # go deeper.
                parent = current
                current = TreeNode(data, level=levels, parent=parent)
                parent.children.append(current)
            elif levels < current.level:
                # backtrack
                while parent is not None and levels - 1 != parent.level:
                    parent = parent.parent
                # at this point, either parent is None, or parent.level == levels - 1 => corrent parent of i
                assert parent is not None
                current = TreeNode(data, level=levels, parent=parent)
                parent.children.append(current)

        # backtrack to root.
        root = parent
        while root.parent is not None:
            root = root.parent
        # root.parent is None -- found the root.

        longest = 0
        # traverse the tree using bfs.
        q = [root]
        while len(q):
            cur = q.pop(0)
            if '.' in cur.data and cur.length + cur.level > longest:
                longest = cur.length + cur.level
            # print('\t'*cur.level + cur.data + '  len=' + str(cur.length))
            q = cur.children + q  # treat as a stack
        return longest

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [2]:
# %load test_length_longest_path.py
from nose.tools import assert_equal, assert_raises


class TestSolution(object):

    def test_length_longest_path(self):
        solution = Solution()
        assert_raises(TypeError, solution.length_longest_path, None)
        assert_equal(solution.length_longest_path(''), 0)
        file_system = 'dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext'
        expected = 32
        assert_equal(solution.length_longest_path(file_system), expected)
        print('Success: test_length_longest_path')


def main():
    test = TestSolution()
    test.test_length_longest_path()


if __name__ == '__main__':
    main()

Success: test_length_longest_path


## Solution Notebook

Review the [Solution Notebook]() for a discussion on algorithms and code solutions.